In [1]:
# Step 1: Import Libraries and Load the Model
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import torch
import torch.nn as nn

# Define the PyTorch model class (must match the training notebook)
class SimpleRNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleRNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        embedded = self.embedding(x)
        rnn_out, hidden = self.rnn(embedded)
        output = self.fc(rnn_out[:, -1, :])
        output = self.sigmoid(output)
        return output

In [2]:

# Load the IMDB dataset word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

In [3]:
# Load the pre-trained PyTorch model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model parameters (must match training parameters)
vocab_size = 10000
embedding_dim = 128
hidden_dim = 128
output_dim = 1

# Create model instance first, then load the state dictionary
model = SimpleRNNClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)
model.load_state_dict(torch.load('simple_rnn_imdb_pytorch.pth', map_location=device))
model.to(device)
model.eval()

print("✅ PyTorch model loaded successfully!")

Using device: cpu
✅ PyTorch model loaded successfully!


In [4]:
# Display model parameters (PyTorch equivalent of model.get_weights())
print("Model Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Show sample weights
print(f"\nEmbedding layer weight shape: {model.embedding.weight.shape}")
print(f"RNN weight shape: {model.rnn.weight_ih_l0.shape}")
print(f"FC layer weight shape: {model.fc.weight.shape}")

Model Architecture:
SimpleRNNClassifier(
  (embedding): Embedding(10000, 128)
  (rnn): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Total parameters: 1,313,153
Trainable parameters: 1,313,153

Embedding layer weight shape: torch.Size([10000, 128])
RNN weight shape: torch.Size([128, 128])
FC layer weight shape: torch.Size([1, 128])


In [5]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input (updated for PyTorch)
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    # Convert to PyTorch tensor
    return torch.LongTensor(padded_review).to(device)

In [6]:
### Prediction function (updated for PyTorch)

def predict_sentiment(review):
    model.eval()  # Ensure model is in evaluation mode
    
    with torch.no_grad():  # Disable gradient computation for inference
        preprocessed_input = preprocess_text(review)
        prediction = model(preprocessed_input)  # Use PyTorch forward pass
        
        # Convert to numpy for easier handling
        prediction_score = prediction.cpu().numpy()[0][0]
        
        sentiment = 'Positive' if prediction_score > 0.5 else 'Negative'
        
        return sentiment, prediction_score

In [7]:
# Step 4: User Input and Prediction
# Example review for prediction
example_review = "This movie was fantastic! The acting was great and the plot was thrilling."

sentiment, score = predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score:.4f}')
print(f'Confidence: {abs(score - 0.5) * 200:.1f}%')

Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.8279
Confidence: 65.6%


In [8]:
# Display model parameters (PyTorch equivalent of model.get_weights())
print("Model parameters:")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

# Show model architecture
print(f"\nModel architecture:\n{model}")

# Show sample weights
print(f"\nEmbedding layer weight shape: {model.embedding.weight.shape}")
print(f"RNN weight shape: {model.rnn.weight_ih_l0.shape}")
print(f"FC layer weight shape: {model.fc.weight.shape}")

Model parameters:
Total parameters: 1,313,153

Model architecture:
SimpleRNNClassifier(
  (embedding): Embedding(10000, 128)
  (rnn): RNN(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Embedding layer weight shape: torch.Size([10000, 128])
RNN weight shape: torch.Size([128, 128])
FC layer weight shape: torch.Size([1, 128])


In [9]:
# Step 2: Helper Functions
# Function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

# Function to preprocess user input (updated for PyTorch)
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500)
    # Convert to PyTorch tensor
    return torch.LongTensor(padded_review).to(device)